In [13]:
![ ! -f requirements.txt ] && git clone https://github.com/anudaweerasinghe/small-coder.git

In [ ]:
! pip3 install -r requirements.txt 

In [10]:
# setup human eval and mbpp dataset
from datasets import load_dataset
humaneval = load_dataset("openai_humaneval", split="test")
mbpp = load_dataset("mbpp", split="test")

Generating prompt split: 100%|██████████| 10/10 [00:00<00:00, 5376.62 examples/s]


In [12]:
print(humaneval[0]['prompt'])
print(mbpp[0]['text'])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

Write a python function to remove first and last occurrence of a given character from the string.


In [ ]:
#initialize deepsek-coder-1.3b
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()

In [ ]:
for datum in humaneval:
  messages=[
    { 'role': 'user', 'content': humaneval['prompt']}
  ]